# Part 3: Evaluate Geometrical Brownian Motion Parameters

**Notebook is under cleaning**

Import libraries

In [1]:
import os
import pickle

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from src.gbm import (drift_sigma_maxlikelihood, compute_gbm_params, fit_drift_skewnormal, fit_volatility_gamma,
                     plot_gbm_parameter_distribution, linear_regression_fit, plot_kde, plot_drift_vs_sigma_fit)

# PATH TO DATA
FILEPATH = "data/all_indexes_2006-01-01_2021-12-31.pickle"

with open(FILEPATH, "rb") as f:
    data = pickle.load(f)

START_YEAR = 2006
END_YEAR = 2021

T = END_YEAR - START_YEAR

Define groups of indexes:

In [2]:
# Unites States indexes
indexes_usa = ['SPX','CCMP','RIY','RTY','RAY','RLV','RLG','NBI']

# S&P500 sectors
indexes_sp500 = ['S5COND','S5CONS','S5ENRS','S5FINL','S5HLTH','S5INFT','S5MATR','S5TELS','S5UTIL','S5INDU']

# European indexes
indexes_eu = ['DAX','CAC','UKX','BEL20','IBEX','KFX','OMX','SMI']

# Asia Pacific indexes
indexes_apac = ['AS51'] 

# Japan indexes
indexes_jp = ['NKY','TPX']

# BRIC indexes
indexes_bric = ['IBOV','NIFTY','MXIN','SHCOMP','SHSZ300'] 

# All indexes: merge all lists
indexes_all = indexes_usa + indexes_sp500 + indexes_eu + indexes_apac + indexes_jp + indexes_bric

index_list = ['SPX']

Compute GBM parameters and fit them with linear function.

In [3]:
results_fit = []

data_index = {}

for index_name in indexes_all:
    
    print(f"Processing {index_name} index")
    
    df = compute_gbm_params(data, index_name)

    ## === Drift vs sigma (volatility) === ##
    ## The scatter plot does not show any clear pattern
    a, b, r2, corr = linear_regression_fit(df = df, index_name=index_name, fit_intercept = True, huber=True)

    data_index[index_name] = {'a': a, 'b': b, 'R2': r2, 'corr': corr}

    ## === Drift and sigma KDE plots === #
    title = 'GBM drift distribution. Index {}'.format(index_name)
    plot_kde(df.drift, title, 'drift', index_name)

    title = 'GBM volatility distribution. Index {}'.format(index_name)
    plot_kde(df.sigma, title, 'volatility', index_name) 

    ## === Skenormal fit of the drift distribution === ##
    drift_skewnorm = fit_drift_skewnormal(df.drift)
    volatility_gamma = fit_volatility_gamma(df.sigma)

    data_index[index_name].update(drift_skewnorm)
    data_index[index_name].update(volatility_gamma)

    #model_stats = {'model_mean_to_median': np.exp(0.5*volatility_gamma['sigma_mode']**2*T + 0.5*drift_skewnorm['mu_std']**2*T**2),
    #                'model_mean_to_mode': np.exp(3./2.*volatility_gamma['sigma_mode']**2*T + drift_skewnorm['mu_std']**2*T**2),
    #                'model_mean_rho': np.exp(drift_skewnorm['mu_mean']*T + 0.5*drift_skewnorm['mu_std']**2*T**2),
    #                'model_median_rho': np.exp(drift_skewnorm['mu_mean']*T - 0.5*volatility_gamma['sigma_mode']**2*T)}
    #data_index[index_name].update(model_stats)

    corrcoeff = np.corrcoef(np.array(df.drift, dtype=float), np.array(df.sigma, dtype=float))[0,1]
    data_index[index_name].update({'corr_drift_sigma': corrcoeff})

    plot_gbm_parameter_distribution(df.drift, index_name, 'drift')
    plot_gbm_parameter_distribution(df.sigma, index_name, 'volatility')

df_results = pd.DataFrame.from_dict(data_index, orient='index')

Processing SPX index


KeyError: 'Close'

In [ ]:
# columns to be displayed in the paper
columns_paper = ['drift_mean','drift_std','drift_loc','drift_scale','drift_skewp','sigma_mean','sigma_alpha','sigma_beta','a','b','R2','corr_drift_sigma']

# select columns to be displayed in the paper
df_paper = df_results[columns_paper]

df_paper.round(2)

In [ ]:
df_paper.to_csv('index_mu_sigma_distribution_paper_version.csv',index=False)
df_paper.round(2).to_latex(index=False)